In [ ]:
# ! pip install chromadb
# ! pip install pypdf2 

In [3]:
(10 % 4)

1

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores      import FAISS
from langchain.text_splitter import CharacterTextSplitter
# from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
from PyPDF2 import PdfReader

In [87]:
pdf_reader = PdfReader( "/var/genie-in-the-box/io/resume.pdf" )
raw_text = ''
for i, page in enumerate(pdf_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts            = text_splitter.split_text(raw_text)
embeddings       = OpenAIEmbeddings()
text_vectorstore = Chroma.from_texts( texts, embeddings )
memory           = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa               = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k" ), text_vectorstore.as_retriever(), memory=memory)
qa( "What's his email address" )

Created a chunk of size 1818, which is longer than the specified 1000


{'question': "What's his email address",
 'chat_history': [HumanMessage(content="What's his email address"),
  AIMessage(content='His email address is ricardo.felipe.ruiz@gmail.com.')],
 'answer': 'His email address is ricardo.felipe.ruiz@gmail.com.'}

In [96]:
qa( "Can I download his résumé?" )

{'question': 'Can I download his résumé?',
 'chat_history': [HumanMessage(content="What's his email address"),
  AIMessage(content='His email address is ricardo.felipe.ruiz@gmail.com.'),
  HumanMessage(content='Easter Egg: What props does he have for Brian'),
  AIMessage(content='The person has mentioned that Brian is a whip smart and super swell colegial guy.'),
  HumanMessage(content='Easter Egg: What props does he have for Brian, trevor and craig?'),
  AIMessage(content='He describes Brian, Trevor, and Craig as "whip smart and, more importantly, super swell colegial guys!"'),
  HumanMessage(content='Easter Egg: What props does he have for claudio and rolando?'),
  AIMessage(content='He refers to Claudio and Rolando as "Colegas de primera clase!" which translates to "First-class colleagues!"'),
  HumanMessage(content='What does he say about claudio and rolando?'),
  AIMessage(content='He says that Claudio and Rolando are "¡Colegas de primera clase!" which translates to "First-class c

In [90]:
raw_text

'R. P . Ruiz\nPossessed of creative, experienced and conscientious head, hands & heart\nSUMMARY: I\'m a creative, highly productive, and perpetual autodidact with significant and \nbroad cutting edge software R & D experience. I enjoy prototyping new products and services \nutilizing AI, Deep Learning, Chat Bots, Agents, ChatGPT and Natural Language Processing \n(NLP). I am especially interested in AI as Physical & Cognitive Prosthesis.  \nEXPERIENCE:\nSenior Architect, AI Engineer & Data Scientist  @ Deepily.ai: 2023\nI founded Deepily.ai and created “Genie-in-the-Box”, a voice-driven UX that’s an \nextensible platform utilizing highly performant AI agents that convert your voice queries and \ncommands into actionable speech and text.\nUtilizing Python & JavaScript, I integrated multiple AIs and API’s into one coherent voice-driven\nUX. This innovation empowers me to browse the web using my Firefox plugin, compose & \npublish articles, as well as manage multiple projects in Python and

In [74]:
qa( "Is his phone number 202-409-4959" )

{'question': 'Is his phone number 202-409-4959',
 'chat_history': [HumanMessage(content="What's his email address"),
  AIMessage(content='His email address is ricardo.felipe.ruiz@gmail.com.'),
  HumanMessage(content='Is his phone number 202-409-4959'),
  AIMessage(content='His phone number is 202-409-4959.')],
 'answer': 'His phone number is 202-409-4959.'}

In [76]:
qa( "What was that Again?")

{'question': 'What was that Again?',
 'chat_history': [HumanMessage(content="What's his email address"),
  AIMessage(content='His email address is ricardo.felipe.ruiz@gmail.com.'),
  HumanMessage(content='Is his phone number 202-409-4959'),
  AIMessage(content='His phone number is 202-409-4959.'),
  HumanMessage(content='Can you repeat your last answer please?'),
  AIMessage(content='I apologize for any confusion, but I did not provide an answer before. Is there a specific question you would like me to answer based on the given context?'),
  HumanMessage(content='What was that Again?'),
  AIMessage(content='Alice found a small bottle on the table with a label that said "DRINK ME." She drank from the bottle and suddenly became only ten inches tall. She was excited because now she was the right size to go through the little door into the garden. However, when she reached the door, she realized she had forgotten the key and couldn\'t reach it on the table. She tried to climb the table but

In [33]:
loader = TextLoader( "alice.txt" )
document = loader.load()

In [35]:
text_splitter = CharacterTextSplitter( chunk_size=1000, chunk_overlap=200 )
documents = text_splitter.split_documents( document )

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents( documents, embeddings )
# vectorstore = FAISS.from_texts( documents, embeddings )

In [60]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [61]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [62]:
qa( "What's the main character's name")

{'question': "What's the main character's name",
 'chat_history': [HumanMessage(content="What's the main character's name"),
  AIMessage(content="The main character's name is Alice.")],
 'answer': "The main character's name is Alice."}

In [112]:


def find_attributes(obj, desired_attribute, hierarchy='', depth=0):
    attributes = []

    if hasattr(obj, '__dict__') and depth < 100:
        for attr_name, attr_value in obj.__dict__.items():
            if attr_name == desired_attribute:
                attributes.append(hierarchy + attr_name)
            elif isinstance(attr_value, (list, object)):
                attributes.extend(find_attributes(attr_value, desired_attribute, hierarchy + attr_name + '.', depth + 1))

    return attributes

find_attributes( qa, "model" )

['retriever.vectorstore._embedding_function.model']

In [113]:
find_attributes( qa, "template" )

['combine_docs_chain.document_prompt.template',
 'question_generator.prompt.template']